In [1]:
import pandas as pd
import re

In [2]:
user = pd.read_csv('D:\\social networks\\user.csv')
movie = pd.read_csv('D:\\social networks\\movie.csv') 

## 将user数据集中的评论字典化 

In [3]:
comment_dict = []
for i in range(len(user.comments)):
    str = user.comments[i]
    movie_id = re.findall(r"[{,](.+?)[:]",str)
    comments = re.findall(r"[:](.+?)[！} ,]",str)
    comment_d = dict(zip(movie_id, comments))  
    comment_dict.append(comment_d)

In [4]:
user['comment_dict'] = comment_dict

## 将user数据集中的rates字典化,并把所打分数转化为数值型（float） 

In [5]:
rates_dict = []
movie_id = []
for i in range(len(user.comments)):
    str = user.rates[i]
    movie_id = re.findall(r"{u'(.+?)[': ]",str)
    movie_id_s = re.findall(r", u'(.+?)[': ]",str)
    movie_id.extend(movie_id_s)
    rate = re.findall(r": u'(.+?)'",str)
    rate_float =  list(map(float,rate))#分数由字符型转为数值型
    rates_d = dict(zip(movie_id, rate_float))  
    rates_dict.append(rates_d)

In [6]:
user['rates_dict'] = rates_dict

## 将user数据集中的name、following_id做出gephi的输入集

In [12]:
k = 0
fllowing_id_list = []
name_source = []
following_target = []
for i in range(len(user.following_id)):
    str = user.following_id[i]
    fllowing_id_list = re.findall(r"u'(.+?)'",str)
    for j in range(len(fllowing_id_list)):
        if fllowing_id_list[j] in set(user.name):
            name_source.append(user.name[i])
            following_target.append(fllowing_id_list [j])
            k = k + 1 
        else:
            continue

In [13]:
input_dict = {
    "Source":name_source,
    "Target":following_target
}
gephi_input = pd.DataFrame(input_dict)

输出gephi要用的表

In [10]:
gephi_input.to_csv('gephi_input.csv',index=False,  header=True)

## Surprise库所需要的数据：[user_id, product_id, rating]类型的数据——爬虫前

In [14]:
k = 0
user_id_list = []
movie_id_list  = []
rate_list = []
movie_id = []
for i in range(len(user.rates)):
    str = user.rates[i]
    movie_id = re.findall(r"{u'(.+?)[': ]",str)
    movie_id_s = re.findall(r", u'(.+?)[': ]",str)
    movie_id.extend(movie_id_s)
    rate = re.findall(r": u'(.+?)'",str)
    rate_float =  list(map(float,rate))#分数由字符型转为数值型
    for j in range(len(movie_id)):
        if int(movie_id[j]) in set(movie.id):
            user_id_list.append(user.user_id[i])
            movie_id_list.append(movie_id[j])
            rate_list.append(rate_float[j])
            k = k + 1 
        else:
            continue

In [15]:
input_dict = {
    "user_id":user_id_list,
    "movie_id":movie_id_list,
    "rates":rate_list
}
Surprise_input = pd.DataFrame(input_dict)

In [16]:
Surprise_input.to_csv('Surprise_input.csv',index=False,  header=True)